<a href="https://colab.research.google.com/github/ICPCfirst/DeepLearningHW2/blob/main/HW2(Generative).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

np.random.seed(0)
X_train_fpath = "content/drive/MyDrive/data/X_train"
X_test_fpath = "content/drive/MyDrive/data/X_test"
Y_train_fpath = "content/drive/MyDrive/data/Y_train"
output_fpath = "./output_{}.csv"

In [2]:
with open(X_train_fpath) as f:
  next(f)
  X_train = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
with open(X_test_fpath) as f:
  next(f)
  X_test = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
with open(Y_train_fpath) as f:
  next(f)
  Y_train = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
  

In [3]:
def _normalize(X,train = True,specified_column = None,X_mean = None,X_std = None):
  if specified_column == None:
    specified_column = np.arange(X.shape[1])
  if train:
    X_mean = np.mean(X[:,specified_column],0).reshape(1,-1) # meean(0)==压缩行 ；reshape == 必须转化为一行n列 ？？？？？？？是不是重复了
    X_std = np.std(X[:,specified_column]).reshape(1,-1)
  
  X[:,specified_column] = (X[:,specified_column]-X_mean)/(X_std + 1e-8)# 进行归一化，中心化和标准处理化
  return X,X_mean,X_std

In [4]:
def _train_dev_split(X,Y,dev_ratio = 0.25):
  train_size = int(len(X)*(1-dev_ratio))
  return X[:train_size],Y[:train_size],X[train_size:],Y[train_size:]

In [5]:
X_train,X_mean,X_std = _normalize(X_train,train = True)
X_test,_,_ = _normalize(X_test,train = False,specified_column=None,X_mean = X_mean,X_std = X_std)

dev_ratio = 0.1
X_train, Y_train, X_dev, Y_dev = _train_dev_split(X_train, Y_train, dev_ratio = dev_ratio)

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]


In [6]:
def _shuffle(X, Y):
    # This function shuffles two equal-length list/array, X and Y, together.
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
    return (X[randomize], Y[randomize])

def _sigmoid(z):
    # Sigmoid function can be used to calculate probability.
    # To avoid overflow, minimum/maximum output value is set.
    return np.clip(1 / (1.0 + np.exp(-z)), 1e-8, 1 - (1e-8))

def _f(X, w, b):
    # This is the logistic regression function, parameterized by w and b
    #
    # Arguements:
    #     X: input data, shape = [batch_size, data_dimension]
    #     w: weight vector, shape = [data_dimension, ]
    #     b: bias, scalar
    # Output:
    #     predicted probability of each row of X being positively labeled, shape = [batch_size, ]
    return _sigmoid(np.matmul(X, w) + b)

def _predict(X, w, b):
    # This function returns a truth value prediction for each row of X 
    # by rounding the result of logistic regression function.
    return np.round(_f(X, w, b)).astype(np.int)
    
def _accuracy(Y_pred, Y_label):
    # This function calculates prediction accuracy
    acc = 1 - np.mean(np.abs(Y_pred - Y_label))
    return acc


In [7]:
# 分别计算类别0和类别1的均值
X_train_0 = np.array([x for x, y in zip(X_train, Y_train) if y == 0])  # u0，内容全为X
X_train_1 = np.array([x for x, y in zip(X_train, Y_train) if y == 1])  # u1

mean_0 = np.mean(X_train_0, axis = 0) # 压缩行，求各列均值
mean_1 = np.mean(X_train_1, axis = 0)

#print(X_train_0.shape) #(38784, 510) 有三万多行数据的结果是u0
#print("*"*50) 
#print(X_train_1.shape) #(10046, 510)) 有一万多行数据的结果是u1
#print("*"*50)
#print(mean_0)
#print("*"*50)
#print(mean_1.shape)

In [8]:
# 分别计算类别0和类别1的协方差
cov_0 = np.zeros((data_dim, data_dim))
cov_1 = np.zeros((data_dim, data_dim))

for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0] #求每一个判断依据（如是否上过大学）的Covariance（就是E)
for x in X_train_1:
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / X_train_1.shape[0]

# 共享协方差 = 独立的协方差的加权求和 就是E
cov = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])

# 计算协方差矩阵的逆
# 协方差矩阵可能是奇异矩阵, 直接使用np.linalg.inv() 可能会产生错误
# 通过SVD矩阵分解，可以快速准确地获得方差矩阵的逆
# A(m,n)=U(m,r)Σ(r,r)V(r,n)T  我们仅需保留着三个比较小的矩阵，就能表示A，不仅节省存储量，在计算的时候更是减少了计算量  SVD
u, s, v = np.linalg.svd(cov, full_matrices=False) #emmm,以后。。。再补注释，反正现在直接用公式就好
inv = np.matmul(v.T * 1 / s, u.T)

# 计算w和b
w = np.dot(inv, mean_0 - mean_1)
b =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_1, np.dot(inv, mean_1))\
    + np.log(float(X_train_0.shape[0]) / X_train_1.shape[0]) 

In [ ]:
# 计算训练集上的准确率
Y_train_pred = 1 - _predict(X_train, w, b)
print('Training accuracy: {}'.format(_accuracy(Y_train_pred, Y_train)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
